# Data Sentiment Classification

Useful links:
    * https://pypi.org/project/langdetect/ 
    * https://pypi.org/project/pydeepl/

### Imports

In [3]:
!pip install langdetect

  Using cached https://files.pythonhosted.org/packages/56/a3/8407c1e62d5980188b4acc45ef3d94b933d14a2ebc9ef3505f22cf772570/langdetect-1.0.8.tar.gz
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993198 sha256=10a4be95ebba6dab66df37f8ca9393fc1d3712fc1544ef20c044af4b8ef7152b
  Stored in directory: C:\Users\salim\AppData\Local\pip\Cache\wheels\8d\b3\aa\6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [13]:
!pip install pydeepl

  Created wheel for pydeepl: filename=pydeepl-0.13-cp37-none-any.whl size=4925 sha256=2d4a6f3ba24844dcc2f9b2ab8ce2d0d58f728bdf15c220ed7754456826d3d676
  Stored in directory: C:\Users\salim\AppData\Local\pip\Cache\wheels\ee\f0\61\6983aa6a1b326b371253abf20425aa43bfa277e1a97390bd47
Successfully built pydeepl


In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from collections import Counter
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect,DetectorFactory
DetectorFactory.seed = 0 # Keep deterministic tranlation for ambiguous sentences
import pydeepl
from transformers import pipeline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\salim\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Load predictions 

In [2]:
DATA_PATH = "data"

test_df = pd.read_csv(DATA_PATH+"/vaccination_tweets.csv")
#test_df = test_df.set_index('Id',drop=False)
test_df.index.name = None
test_df.head(2)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False


In [3]:
def predict_vader_sentiment(vader_score, threshold):
    if vader_score["compound"] >= threshold :
        sentiment = 1 
    elif vader_score["compound"] <=-threshold:
        sentiment = -1
    else:
        sentiment = 0
    return sentiment

In [4]:
def predict_trans_sentiment(pred_dict,threshold):
    threshold = 1 - threshold
    if pred_dict["score"] >= threshold and pred_dict["label"]=="POSITIVE"  :
        sentiment = 1 
    elif pred_dict["score"] >= threshold and pred_dict["label"]=="NEGATIVE":
        sentiment = -1
    else:
        sentiment = 0
    return sentiment
    

In [10]:
vader_sid = SentimentIntensityAnalyzer()
transf_classifier = pipeline('sentiment-analysis')
threshold = 0.05

vader_scores=[]
vader_sentiments = []
transf_scores = []
transf_sentiments = []

for text in test_df['text']:    
    lang = detect(text)
    if lang != 'en':
        try:
            text = pydeepl.translate(text,"EN")
        except:
            pass
    # predict sentiment with vader 
    vader_score= vader_sid.polarity_scores(text)
    vader_scores.append(vader_score["compound"])
    vader_sentiment = predict_vader_sentiment(vader_score, threshold)
    vader_sentiments.append(vader_sentiment)
    
    # predict sentiment with transformers
    pred_dict = transf_classifier(text)
    print(pred_dict)
    transf_scores.append(pred_dict[0]['score'])
    transf_sentiment = predict_trans_sentiment(pred_dict[0],threshold)
    transf_sentiments.append(transf_sentiment)
    

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertModel: ['pre_classifier', 'classifier', 'dropout_19']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.
Some layers from the model checkpoint at distilbert-b

[{'label': 'NEGATIVE', 'score': 0.9946713447570801}]
[{'label': 'POSITIVE', 'score': 0.896996021270752}]
[{'label': 'NEGATIVE', 'score': 0.9975707530975342}]
[{'label': 'POSITIVE', 'score': 0.991213858127594}]
[{'label': 'NEGATIVE', 'score': 0.9955069422721863}]
[{'label': 'NEGATIVE', 'score': 0.998723030090332}]
[{'label': 'NEGATIVE', 'score': 0.9983234405517578}]
[{'label': 'POSITIVE', 'score': 0.9995377063751221}]
[{'label': 'NEGATIVE', 'score': 0.9905767440795898}]
[{'label': 'NEGATIVE', 'score': 0.9960853457450867}]
[{'label': 'NEGATIVE', 'score': 0.9973651766777039}]
[{'label': 'NEGATIVE', 'score': 0.9923699498176575}]
[{'label': 'NEGATIVE', 'score': 0.982143223285675}]
[{'label': 'NEGATIVE', 'score': 0.9796897768974304}]
[{'label': 'NEGATIVE', 'score': 0.9949651956558228}]
[{'label': 'NEGATIVE', 'score': 0.9668165445327759}]
[{'label': 'NEGATIVE', 'score': 0.9893110990524292}]
[{'label': 'NEGATIVE', 'score': 0.9896165132522583}]
[{'label': 'NEGATIVE', 'score': 0.9949198961257935

[{'label': 'NEGATIVE', 'score': 0.9649260640144348}]
[{'label': 'POSITIVE', 'score': 0.9614360332489014}]
[{'label': 'NEGATIVE', 'score': 0.991630494594574}]
[{'label': 'NEGATIVE', 'score': 0.9926949739456177}]
[{'label': 'NEGATIVE', 'score': 0.9004947543144226}]
[{'label': 'NEGATIVE', 'score': 0.9968101382255554}]
[{'label': 'NEGATIVE', 'score': 0.6997156739234924}]
[{'label': 'POSITIVE', 'score': 0.9948594570159912}]
[{'label': 'NEGATIVE', 'score': 0.9811925888061523}]
[{'label': 'NEGATIVE', 'score': 0.9865508079528809}]
[{'label': 'POSITIVE', 'score': 0.9917919039726257}]
[{'label': 'POSITIVE', 'score': 0.9984680414199829}]
[{'label': 'NEGATIVE', 'score': 0.6329125761985779}]
[{'label': 'POSITIVE', 'score': 0.8460646867752075}]
[{'label': 'POSITIVE', 'score': 0.9787138104438782}]
[{'label': 'NEGATIVE', 'score': 0.9934740662574768}]
[{'label': 'NEGATIVE', 'score': 0.9956130385398865}]
[{'label': 'NEGATIVE', 'score': 0.9965404272079468}]
[{'label': 'NEGATIVE', 'score': 0.9994745254516

[{'label': 'NEGATIVE', 'score': 0.9943273663520813}]
[{'label': 'NEGATIVE', 'score': 0.989230215549469}]
[{'label': 'POSITIVE', 'score': 0.9969704747200012}]
[{'label': 'NEGATIVE', 'score': 0.9950202107429504}]
[{'label': 'NEGATIVE', 'score': 0.990475594997406}]
[{'label': 'NEGATIVE', 'score': 0.9957378506660461}]
[{'label': 'NEGATIVE', 'score': 0.9951623678207397}]
[{'label': 'NEGATIVE', 'score': 0.9953597784042358}]
[{'label': 'POSITIVE', 'score': 0.9533447027206421}]
[{'label': 'NEGATIVE', 'score': 0.9966701865196228}]
[{'label': 'NEGATIVE', 'score': 0.9987015128135681}]
[{'label': 'NEGATIVE', 'score': 0.996652364730835}]
[{'label': 'NEGATIVE', 'score': 0.9987381100654602}]
[{'label': 'NEGATIVE', 'score': 0.9996153116226196}]
[{'label': 'POSITIVE', 'score': 0.9985578060150146}]
[{'label': 'POSITIVE', 'score': 0.9925885200500488}]
[{'label': 'NEGATIVE', 'score': 0.9948322772979736}]
[{'label': 'NEGATIVE', 'score': 0.9921578764915466}]
[{'label': 'NEGATIVE', 'score': 0.965245068073272

[{'label': 'POSITIVE', 'score': 0.995925784111023}]
[{'label': 'POSITIVE', 'score': 0.8696057200431824}]
[{'label': 'POSITIVE', 'score': 0.9918786287307739}]
[{'label': 'NEGATIVE', 'score': 0.9643082618713379}]
[{'label': 'NEGATIVE', 'score': 0.9608849883079529}]
[{'label': 'NEGATIVE', 'score': 0.9987853169441223}]
[{'label': 'NEGATIVE', 'score': 0.9910188317298889}]
[{'label': 'NEGATIVE', 'score': 0.9990241527557373}]
[{'label': 'NEGATIVE', 'score': 0.992418646812439}]
[{'label': 'POSITIVE', 'score': 0.998957633972168}]
[{'label': 'NEGATIVE', 'score': 0.9787809252738953}]
[{'label': 'POSITIVE', 'score': 0.9098794460296631}]
[{'label': 'NEGATIVE', 'score': 0.783955454826355}]
[{'label': 'NEGATIVE', 'score': 0.9934625029563904}]
[{'label': 'NEGATIVE', 'score': 0.9975771307945251}]
[{'label': 'NEGATIVE', 'score': 0.9904327988624573}]
[{'label': 'POSITIVE', 'score': 0.8164964318275452}]
[{'label': 'NEGATIVE', 'score': 0.9987671375274658}]
[{'label': 'NEGATIVE', 'score': 0.9962159395217896

[{'label': 'NEGATIVE', 'score': 0.9854816198348999}]
[{'label': 'NEGATIVE', 'score': 0.9386357665061951}]
[{'label': 'NEGATIVE', 'score': 0.975662112236023}]
[{'label': 'NEGATIVE', 'score': 0.9815136194229126}]
[{'label': 'NEGATIVE', 'score': 0.9113754630088806}]
[{'label': 'NEGATIVE', 'score': 0.995843231678009}]
[{'label': 'NEGATIVE', 'score': 0.9947148561477661}]
[{'label': 'POSITIVE', 'score': 0.9842343926429749}]
[{'label': 'NEGATIVE', 'score': 0.9935449361801147}]
[{'label': 'NEGATIVE', 'score': 0.9906129837036133}]
[{'label': 'POSITIVE', 'score': 0.9996868371963501}]
[{'label': 'NEGATIVE', 'score': 0.9976081252098083}]
[{'label': 'NEGATIVE', 'score': 0.9965147376060486}]
[{'label': 'NEGATIVE', 'score': 0.9569434523582458}]
[{'label': 'POSITIVE', 'score': 0.9987149238586426}]
[{'label': 'NEGATIVE', 'score': 0.9946747422218323}]
[{'label': 'NEGATIVE', 'score': 0.9963955283164978}]
[{'label': 'NEGATIVE', 'score': 0.9962661862373352}]
[{'label': 'NEGATIVE', 'score': 0.97221446037292

[{'label': 'POSITIVE', 'score': 0.6020139455795288}]
[{'label': 'NEGATIVE', 'score': 0.9968813061714172}]
[{'label': 'NEGATIVE', 'score': 0.9889323711395264}]
[{'label': 'NEGATIVE', 'score': 0.993827760219574}]
[{'label': 'NEGATIVE', 'score': 0.9829596281051636}]
[{'label': 'NEGATIVE', 'score': 0.9909693002700806}]
[{'label': 'NEGATIVE', 'score': 0.9469330310821533}]
[{'label': 'NEGATIVE', 'score': 0.9994799494743347}]
[{'label': 'NEGATIVE', 'score': 0.9644506573677063}]
[{'label': 'NEGATIVE', 'score': 0.9957453012466431}]
[{'label': 'NEGATIVE', 'score': 0.9979770183563232}]
[{'label': 'NEGATIVE', 'score': 0.994063675403595}]
[{'label': 'POSITIVE', 'score': 0.9570499062538147}]
[{'label': 'NEGATIVE', 'score': 0.9878603219985962}]
[{'label': 'NEGATIVE', 'score': 0.9896933436393738}]
[{'label': 'NEGATIVE', 'score': 0.9775967597961426}]
[{'label': 'POSITIVE', 'score': 0.9984233379364014}]
[{'label': 'NEGATIVE', 'score': 0.9938589930534363}]
[{'label': 'POSITIVE', 'score': 0.97583675384521

[{'label': 'NEGATIVE', 'score': 0.9975960850715637}]
[{'label': 'NEGATIVE', 'score': 0.9408459663391113}]
[{'label': 'NEGATIVE', 'score': 0.9841064214706421}]
[{'label': 'NEGATIVE', 'score': 0.9953364133834839}]
[{'label': 'NEGATIVE', 'score': 0.9972909092903137}]
[{'label': 'NEGATIVE', 'score': 0.9943856000900269}]
[{'label': 'POSITIVE', 'score': 0.6652069091796875}]
[{'label': 'NEGATIVE', 'score': 0.9981856346130371}]
[{'label': 'POSITIVE', 'score': 0.9727256894111633}]
[{'label': 'NEGATIVE', 'score': 0.9942734837532043}]
[{'label': 'NEGATIVE', 'score': 0.9929416179656982}]
[{'label': 'NEGATIVE', 'score': 0.9968745112419128}]
[{'label': 'NEGATIVE', 'score': 0.999366819858551}]
[{'label': 'NEGATIVE', 'score': 0.9919776320457458}]
[{'label': 'NEGATIVE', 'score': 0.9936055541038513}]
[{'label': 'NEGATIVE', 'score': 0.987670361995697}]
[{'label': 'NEGATIVE', 'score': 0.9649974703788757}]
[{'label': 'NEGATIVE', 'score': 0.7540713548660278}]
[{'label': 'POSITIVE', 'score': 0.99922364950180

[{'label': 'NEGATIVE', 'score': 0.9987325072288513}]
[{'label': 'POSITIVE', 'score': 0.8281683921813965}]
[{'label': 'NEGATIVE', 'score': 0.996920645236969}]
[{'label': 'NEGATIVE', 'score': 0.998756468296051}]
[{'label': 'NEGATIVE', 'score': 0.9653708338737488}]
[{'label': 'NEGATIVE', 'score': 0.9937648773193359}]
[{'label': 'NEGATIVE', 'score': 0.9657386541366577}]
[{'label': 'POSITIVE', 'score': 0.5901421904563904}]
[{'label': 'POSITIVE', 'score': 0.9951647520065308}]
[{'label': 'NEGATIVE', 'score': 0.9164415001869202}]
[{'label': 'NEGATIVE', 'score': 0.9861983060836792}]
[{'label': 'POSITIVE', 'score': 0.9987488985061646}]
[{'label': 'NEGATIVE', 'score': 0.9962915778160095}]
[{'label': 'NEGATIVE', 'score': 0.9905275106430054}]
[{'label': 'NEGATIVE', 'score': 0.9944621324539185}]
[{'label': 'POSITIVE', 'score': 0.6349657773971558}]
[{'label': 'POSITIVE', 'score': 0.9969843029975891}]
[{'label': 'NEGATIVE', 'score': 0.9986746907234192}]
[{'label': 'NEGATIVE', 'score': 0.86898869276046

[{'label': 'NEGATIVE', 'score': 0.9910115003585815}]
[{'label': 'NEGATIVE', 'score': 0.995893657207489}]
[{'label': 'POSITIVE', 'score': 0.5534440875053406}]
[{'label': 'NEGATIVE', 'score': 0.9960728883743286}]
[{'label': 'NEGATIVE', 'score': 0.9869154095649719}]
[{'label': 'NEGATIVE', 'score': 0.5045870542526245}]
[{'label': 'NEGATIVE', 'score': 0.9976299405097961}]
[{'label': 'NEGATIVE', 'score': 0.9890116453170776}]
[{'label': 'POSITIVE', 'score': 0.7224386930465698}]
[{'label': 'NEGATIVE', 'score': 0.9969022274017334}]
[{'label': 'NEGATIVE', 'score': 0.9979916214942932}]
[{'label': 'NEGATIVE', 'score': 0.9368616342544556}]
[{'label': 'NEGATIVE', 'score': 0.9860721826553345}]
[{'label': 'NEGATIVE', 'score': 0.9943803548812866}]
[{'label': 'NEGATIVE', 'score': 0.991635262966156}]
[{'label': 'NEGATIVE', 'score': 0.9823795557022095}]
[{'label': 'NEGATIVE', 'score': 0.9762948155403137}]
[{'label': 'NEGATIVE', 'score': 0.9937889575958252}]
[{'label': 'NEGATIVE', 'score': 0.93528431653976

In [11]:
df = pd.DataFrame(test_df["text"], columns = ['text'])
df['vader_scores'] = vader_scores
df['vader_sentiment'] = vader_sentiments
df['transf_scores'] = transf_scores
df['transf_sentiment'] = transf_sentiments
df.head()

,text,vader_scores,vader_sentiment,transf_scores,transf_sentiment
0,Same folks said daikon paste could treat a cyt...,0.4019,1,0.994671,-1
1,While the world has been on the wrong side of ...,-0.1027,-1,0.896996,0
2,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,0.2500,1,0.997571,-1
3,"Facts are immutable, Senator, even when you're...",0.0000,0,0.991214,1
4,Explain to me again why we need a vaccine @Bor...,0.0000,0,0.995507,-1


In [13]:
vader_pred = df['vader_sentiment']
transf_pred = df['transf_sentiment']

In [14]:
vader_pred.head(5)

0    1
1   -1
2    1
3    0
4    0
Name: vader_sentiment, dtype: int64

In [15]:
transf_pred.head(5)

0   -1
1    0
2   -1
3    1
4   -1
Name: transf_sentiment, dtype: int64

### Confusion matrix

In [16]:
confusion_matrix(vader_pred, transf_pred)

array([[356,  23,  18],
       [900,  96,  34],
       [459, 177, 288]], dtype=int64)

### Make final predictions CSV

In [57]:
def GetKey(dictA, val):
    for key, value in dictA.items():
        if val == value:
            return key
    return "key doesn't exist"
    
def make_final_prediction(vader_prediction, transf_prediction):
    predictions = []
    id_list= []
    idx=0
    for vader_pred, transf_pred in zip(vader_prediction, transf_prediction):
        preds=[vader_pred, transf_pred]
        #pred_occurrence = Counter(preds)
        if vader_pred==transf_pred:
            #prediction=dict(pred_occurrence).keys()
            predictions.append(vader_pred)
            id_list.append(idx)
        idx=idx+1
       
    return id_list, predictions

In [66]:
id_list, predictions=make_final_prediction(vader_pred, transf_pred)

In [67]:
new_df =test_df.iloc[id_list]
new_df["pred"]= predictions
new_df.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,pred
7,1337850832256176136,Dr.Krutika Kuppalli,NaN,"ID, Global Health, VHF, Pandemic Prep, Emergin...",2019-03-25 04:14:29,21924,593,7815,True,2020-12-12 20:04:29,There have not been many bright days in 2020 b...,"['BidenHarris', 'Election2020']",Twitter for iPhone,2,22,False,1
18,1337781178665816064,Braderz73🌹#GTTO 🇪🇺 🇬🇧,"Bristol, UK","One of those lefty types, I believe in a faire...",2012-07-24 08:18:51,6430,6292,45007,False,2020-12-12 15:27:42,.@Pfizer will rake in billions from its expens...,['CovidVaccine'],Twitter for Android,3,3,False,-1


In [61]:
predictions_csv = new_df
new_df.to_csv("data/final_prediction.csv", index=False)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [4]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\salim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import re
import unicodedata
from string import digits
from bs4 import BeautifulSoup #Nettoyage d'HTML
from tqdm import tqdm

digits_list = digits

class CleanText:
    def __init__(self):
        french_stopwords = nltk.corpus.stopwords.words('english')
        self.stopwords = [self.remove_accent(sw) for sw in french_stopwords]

        self.stemmer = nltk.stem.SnowballStemmer('english')
    @staticmethod
    def remove_html_code(txt):
        txt = BeautifulSoup(txt, "html.parser", from_encoding='utf-8').get_text()
        return txt
    @staticmethod
    def convert_text_to_lower_case(txt):
        return txt.lower()
    @staticmethod
    def remove_accent(txt):
        return unicodedata.normalize('NFD', txt).encode('ascii', 'ignore').decode("utf-8")
    @staticmethod
    def remove_non_letters(txt):
        return re.sub('[^a-z_]', ' ', txt)
    def remove_stopwords(self, txt):
        return [w for w in txt.split() if (w not in self.stopwords)]
    def get_stem(self, tokens):
        return [self.stemmer.stem(token) for token in tokens]

In [6]:
cleaner = CleanText()
def apply_all_transformation(txt):
    cleaned_txt = cleaner.remove_html_code(txt)   
    lang = detect(cleaned_txt)
    if lang != 'en':
        try:
            cleaned_txt = pydeepl.translate(cleaned_txt,"EN")
        except:
            pass
    cleaned_txt = cleaner.convert_text_to_lower_case(cleaned_txt)
    cleaned_txt = cleaner.remove_accent(cleaned_txt)
    cleaned_txt = cleaner.remove_non_letters(cleaned_txt)
    cleaned_txt = cleaner.remove_stopwords(cleaned_txt)
    cleaned_txt = cleaner.get_stem(cleaned_txt)
    return cleaned_txt

In [7]:
def clean_df_column(dataset, column, cleaned_column):
    dirty_column = dataset[str(column)]
    clean = [" ".join(apply_all_transformation(x)) for x in tqdm(dirty_column)]
    dataset[str(cleaned_column)] = clean

In [8]:
train_df = test_df

In [9]:
clean_df_column(train_df,'text','text_cleaned')

  0%|          | 0/2351 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:185: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
100%|██████████| 2351/2351 [01:59<00:00, 19.63it/s]


In [14]:
tfidf = TfidfVectorizer(
    min_df = 5,
    max_df = 0.95,
    max_features = 1000
)
tfidf.fit(train_df.text_cleaned)
text = tfidf.transform(train_df.text_cleaned)

In [15]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(text)

In [19]:
print(kmeans.cluster_centers_)
import json
with open('centers.json', 'w') as json_file:
    json.dump({"centers": [list(x) for x in kmeans.cluster_centers_]}, json_file)


[[0.00111525 0.00232355 0.00254505 ... 0.00122398 0.0018605  0.00210802]
 [0.         0.         0.         ... 0.00174897 0.00424155 0.        ]
 [0.00154646 0.         0.         ... 0.00654695 0.00227255 0.        ]]


In [ ]:
for text in test_df['text']:    
    lang = detect(text)
    if lang != 'en':
        try:
            text = pydeepl.translate(text,"EN")
        except:
            pass
        
    prediction.append(kmeans.predict(tfidf.transform(text)))
    